In [ ]:
import numpy as np 
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import datetime as dt
from sklearn.metrics import f1_score
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

In [ ]:
smpl_sub = pd.read_csv('sample_submission.csv',index_col='ID')
item = pd.read_csv('items.csv')
s_train = pd.read_csv('sales_train.csv')
s_train['item_cnt_day'] = s_train['item_cnt_day'].astype('int')
# s_train['date'] = pd.to_datetime(s_train['date']).map(dt.datetime.toordinal)
# s_train = s_train.drop(['date','date_block_num'],axis = 1)

In [ ]:
# dt.datetime.toordinal(pd.to_datetime(s_train['date']))
# s_train.loc[2,'date']
# s_train[(s_train['item_cnt_day']==2.0)]

In [ ]:
test = pd.read_csv('test.csv')
test = test.set_index('ID')
it_cat = pd.read_csv('item_categories.csv')
shop = pd.read_csv('shops.csv')
s_train = s_train.drop(['date'],axis = 1)
s_train.head(3)

In [ ]:
item.head(3)

In [ ]:
it_cat.head(3)

In [ ]:
cd = test.copy()
cd['item_cnt_month'] = -1000.9
test

In [ ]:
# test[(test['item_id']==5320)]
# len(cd)
# ef = s_train[['shop_id','item_id','date_block_num','item_price']]
# ef_grp = ef.groupby(['shop_id','item_id'])
# cd

In [ ]:
shop.head(3)

In [ ]:
smpl_sub.head(3)

In [ ]:
# s_train[(s_train['item_cnt_day']>5)]

In [ ]:
# sgroup = s_train.groupby(['shop_id','item_id'])
# tgroup = test.groupby('shop_id')

In [ ]:
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.preprocessing import LabelBinarizer
# type(sgroup.get_group(5))
# tgroup.get_group(1)
# y = LabelBinarizer(ab.item_name)
# print(y)
# ab.corr()
sgroup = s_train.groupby('item_id')
tgroup = test.groupby('shop_id')

In [ ]:
# sgroup.get_group((5,5320))
n = sgroup.groups.keys()

In [ ]:
# for i in range(len(cd)):
#     a = test.loc[i,'shop_id']
#     b = test.loc[i,'item_id']
#     group = sgroup.get_group((a,b))
#     if (a in test.shop_id.unique())&(b in tgroup.get_group(a).item_id):
#         if len(group)>2:
#             neigh = KNeighborsClassifier(n_neighbors = 2)
#             X = preprocessing.StandardScaler().fit_transform(group[['date_block_num','item_price']].values)
#             X_train, X_test, y_train, y_test = train_test_split( X, group['item_cnt_day'].values, test_size=0, shuffle = False)
#             neigh.fit(X_train,y_train)
#             v = neigh.predict([[ef_grp.get_group((a,b))['date_block_num'].max()+1,ef_grp.get_group((a,b))['item_price'].mean()]])
#             cd.loc[i,'item_cnt_month'] = v
# #           print(f1_score(y_test, neigh.predict(X_test)))
# #         elif 
# #         else:
# #           print(f1_score(y_test, group2['item_cnt_day']))

In [ ]:
parameters={'alpha':[1e-8,1e-3,1e-2,1,5,10,20,30,40]}
ridge=Ridge()
for i in range(len(test)):
    a = test.loc[i,'shop_id']
    b = test.loc[i,'item_id']
    if b in n:
        group = sgroup.get_group(b)
        if len(group)>=4:
#             neigh = KNeighborsClassifier(n_neighbors = 4,weights = 'distance')
#             regr = linear_model.LinearRegression()
            ridge_regressor=GridSearchCV(ridge,parameters,scoring='neg_mean_squared_error',cv=4)
            X_train = group[['date_block_num','item_price']].values
#             X_train = preprocessing.StandardScaler().fit_transform(group[['shop_id','item_price']].values)
#             X_train, X_test, y_train, y_test = train_test_split( X, group['item_cnt_day'].values, test_size=0, shuffle = False)
            y_train = group['item_cnt_day'].values
#             neigh.fit(X_train,y_train)
#             regr.fit(X_train,y_train)
            ridge_regressor.fit(X_train,y_train)
            if a in group['shop_id'].unique():
                v = np.round(ridge_regressor.predict([[34, group[group['shop_id']==a]['item_price'].mean()]]))
                if v<-1:
                    v = -1
                cd.loc[i,'item_cnt_month'] = v
                print(v," first ")
            else:
#                 v = np.round(ridge_regressor.predict([[34, group['item_price'].mean()]]))
                cd.loc[i,'item_cnt_month'] = 0
                print(0," second ")
        else:
            cd.loc[i,'item_cnt_month'] = round(group['item_cnt_day'].mean())
            print(round(group['item_cnt_day'].mean())," third ")
#             print(group['item_cnt_day'].mean())
    else:
#         v = s_train[s_train['shop_id']==a]['item_cnt_day'].mean()
        cd.loc[i,'item_cnt_month'] = 0
        print(0," fourth ")

In [ ]:
de = cd[['item_cnt_month']]

In [ ]:
de['item_cnt_month'] = round(de['item_cnt_month'])
de

In [ ]:
de.to_csv('submssn7.csv')

In [ ]:
# de[de['item_cnt_month']==0]

In [ ]:
_